In [1]:
import lightgbm as lgb
import numpy as np
import optuna.integration.lightgbm as olgb
import optuna.logging
import pandas as pd
import psutil

from onnxconverter_common import FloatTensorType
from onnxmltools.convert import convert_lightgbm
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold, train_test_split

optuna.logging.set_verbosity(optuna.logging.WARNING)

C:\Users\yuki\IdeaProjects\PhishDetector\model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 特徴量データの読み込み
features_and_label: pd.DataFrame = pd.read_csv("../feature.csv").drop(columns=["domain"])

# featuresとlabelを分割
features = features_and_label.drop("label", axis=1)
labels = features_and_label[["label"]]

# LightGBMの形式に変換
train_data = lgb.Dataset(
    data=features,
    label=labels,
    feature_name=features.columns.tolist(),
)

In [3]:
params = {
    "num_threads": psutil.cpu_count() - 1,  # スレッド数
    "objective": "binary",  # 二値分類
    "metric": "binary_logloss",
    "boosting_type": "gbdt",
    "verbosity": -1,  # 学習途中の情報を表示しない
    "seed": 42,
}

In [4]:
# Stratified 5-fold CV

cv_scores_columns = ["accuracy", "TPR", "TNR", "FPR", "FNR", "F1", "AUC", "TP", "TN", "FP", "FN"]
history = np.empty((0, 11))

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in kf.split(features, labels):
    _x_train = features.iloc[train_idx, :]  # 学習用特徴量
    _y_train = labels.iloc[train_idx, :]  # 学習用ラベル
    x_test = features.iloc[test_idx, :]  # テスト用特徴量
    y_test = labels.iloc[test_idx, :]  # テスト用ラベル
    x_train, x_valid, y_train, y_valid = train_test_split(_x_train, _y_train, test_size=0.25, stratify=_y_train, random_state=42)

    dtrain = lgb.Dataset(x_train, label=y_train, feature_name=features.columns.tolist())
    dtest = lgb.Dataset(x_test, label=y_test, feature_name=features.columns.tolist())
    dvalid = lgb.Dataset(x_valid, label=y_valid, feature_name=features.columns.tolist())

    # 訓練
    model = olgb.train(
        params=params,
        train_set=dtrain,
        valid_sets=[dtrain, dvalid],
        callbacks=[lgb.early_stopping(100, verbose=False)],
        num_boost_round=10000,
    )

    # 予測
    label_pred_prob = model.predict(x_test)
    label_pred = (label_pred_prob > 0.5).astype(int)

    # 評価指標の計算
    tn, fp, fn, tp = confusion_matrix(y_test, label_pred).flatten()
    acc = accuracy_score(y_test, label_pred)
    tpr = tp / (tp + fn)  # recall
    tnr = tn / (tn + fp)  # specificity
    fpr = fp / (tn + fp)
    fnr = fn / (tp + fn)
    f1 = f1_score(y_test, label_pred)
    auc = roc_auc_score(y_test, label_pred_prob)

    # 評価結果の格納
    history = np.vstack([history, [acc, tpr, tnr, fpr, fnr, f1, auc, tp, tn, fp, fn]])

min_child_samples, val_score: 0.044265: 100%|##########| 5/5 [00:00<00:00, 11.68it/s]


In [5]:
mean = np.mean(history, axis=0)
history_final = np.vstack([history, mean])

df_cv_scores = pd.DataFrame(history_final, columns=cv_scores_columns, index=["1", "2", "3", "4", "5", "平均"])
df_cv_scores

,accuracy,TPR,TNR,FPR,FNR,F1,AUC,TP,TN,FP,FN
1,0.982412,0.984925,0.979899,0.020101,0.015075,0.982456,0.996136,196.0,195.0,4.0,3.0
2,0.989950,0.994975,0.984925,0.015075,0.005025,0.990000,0.998990,198.0,196.0,3.0,1.0
3,0.984925,0.984925,0.984925,0.015075,0.015075,0.984925,0.998687,196.0,196.0,3.0,3.0
4,0.969849,0.964824,0.974874,0.025126,0.035176,0.969697,0.993510,192.0,194.0,5.0,7.0
5,0.994975,0.994975,0.994975,0.005025,0.005025,0.994975,0.999823,198.0,198.0,1.0,1.0
平均,0.984422,0.984925,0.983920,0.016080,0.015075,0.984411,0.997429,196.0,195.8,3.2,3.0


In [6]:
# 訓練
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

dtrain = lgb.Dataset(x_train, label=y_train, feature_name=features.columns.tolist())
dvalid = lgb.Dataset(x_valid, label=y_valid, feature_name=features.columns.tolist())

# 訓練
model = olgb.train(
    params=params,
    train_set=dtrain,
    valid_sets=[dtrain, dvalid],
    callbacks=[lgb.early_stopping(100, verbose=False)],
    num_boost_round=10000,
)

importance = pd.DataFrame(
    model.feature_importance(),
    index=features.columns.tolist(),
    columns=['importance']
).sort_values(by='importance', ascending=False)

importance

min_child_samples, val_score: 0.038950: 100%|##########| 5/5 [00:00<00:00,  9.04it/s]


,importance
TagCountInHeadTag,1619
externalLinkPercentage,790
noDomainInInternalLink,386
samePageLinkCount,312
scriptTagCount,281
iframeTagCount,57
googleAnalytics,46
noTitle,32
invalidKiyaku,22
copy,9


In [7]:
# ONNXへの変換
initial_type = [
    ("X", FloatTensorType([None, 11])),
]
onx = convert_lightgbm(model=model, initial_types=initial_type, zipmap=False)
with open("../onnx_model/lightgbm.onnx", "wb") as f:
    f.write(onx.SerializeToString())

The maximum opset needed by this model is only 9.


In [8]:
import onnx
onnx_model = onnx.load("../onnx_model/lightgbm.onnx")
onnx.checker.check_model(onnx_model)